In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('data.csv')
data = data.drop(["High","Low"	,"Open","Close","Volume"], axis=1)
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data.set_index("Date",inplace=True)

data.head()

,Adj Close
Date,
2022-01-03,181.778397
2022-01-04,179.471344
2022-01-05,174.697418
2022-01-06,171.781143
2022-01-07,171.950928


In [3]:
data['day'] = [i.day for i in data.index]
data['day_name'] = [i.day_name() for i in data.index]
data['day_of_year'] = [i.dayofyear for i in data.index]
data['week_of_year'] = [i.weekofyear for i in data.index]
data['hour'] = [i.hour for i in data.index]
data['is_weekday'] = [i.isoweekday() for i in data.index]
data.head()

,Adj Close,day,day_name,day_of_year,week_of_year,hour,is_weekday
Date,,,,,,,
2022-01-03,181.778397,3,Monday,3,1,0,1
2022-01-04,179.471344,4,Tuesday,4,1,0,2
2022-01-05,174.697418,5,Wednesday,5,1,0,3
2022-01-06,171.781143,6,Thursday,6,1,0,4
2022-01-07,171.950928,7,Friday,7,1,0,5


In [20]:
from pycaret.anomaly import *
s = setup(data,silent=True,n_jobs=-1)

,Description,Value
0,session_id,1573
1,Original Data,"(24, 7)"
2,Missing Values,False
3,Numeric Features,3
4,Categorical Features,4
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(24, 19)"
9,CPU Jobs,-1


In [21]:
models().index.tolist()

['abod',
 'cluster',
 'cof',
 'iforest',
 'histogram',
 'knn',
 'lof',
 'svm',
 'pca',
 'mcd',
 'sod',
 'sos']

In [22]:
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

,Adj Close,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
Date,,,,,,,,,
2022-01-03,181.778397,3,Monday,3,1,0,1,1,0.025488
2022-01-04,179.471344,4,Tuesday,4,1,0,2,0,-0.014538
2022-01-05,174.697418,5,Wednesday,5,1,0,3,0,-0.020794
2022-01-06,171.781143,6,Thursday,6,1,0,4,0,-0.023553
2022-01-07,171.950928,7,Friday,7,1,0,5,0,-0.030819


In [26]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [27]:
evaluate_model(iforest);

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Anomaly TSnE (3d)', 'tsn…

In [24]:
iforest_results[iforest_results['Anomaly'] == 1].head()


,Adj Close,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
Date,,,,,,,,,
2022-01-03,181.778397,3,Monday,3,1,0,1,1,0.025488
2022-01-24,161.414337,24,Monday,24,4,0,1,1,0.003627
2022-01-31,174.557602,31,Monday,31,5,0,1,1,0.016194


In [25]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.line(iforest_results, x=iforest_results.index, y="Adj Close", title='NYC TAXI TRIPS - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [iforest_results.loc[i]['Adj Close'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()